In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
#from future encoders
data = pd.read_csv("Activities.csv")
data.info()
data.head()

label = "Calories" 

train_set, test_set = train_test_split(data, test_size=0.2, random_state=10)


x_tr = train_set.drop(label,axis=1)
y_train = train_set[label].copy()
x_te = test_set.drop(label,axis=1)
y_test = test_set[label].copy()

#drop categories
numericaldata = data.drop()

num_pipeline = Pipeline([
    ('imputer', Imputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])
#OneHotEncoder

num_pipeline.fit_transform(numericaldata)

x_train = full_pipeline.transform(x_tr) #Process training data
x_test = full_pipeline.transform(x_te) #Process test data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 9 columns):
Unnamed: 0              304 non-null int64
Date                    304 non-null object
Calories                304 non-null int64
Steps                   304 non-null int64
Distance                304 non-null float64
Minutessedentary        304 non-null int64
Minuteslightlyactive    304 non-null int64
Minutesfairlyactive     304 non-null int64
Minutesveryactive       304 non-null int64
dtypes: float64(1), int64(7), object(1)
memory usage: 21.5+ KB


ValueError: could not convert string to float: '2018-10-31'

In [ ]:
from sklearn.tree import DecisionTreeRegressor

print("Decision Tree Regressor")
tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(x_train, y_train)

#Decision Tree Regression
#Prediction on training set
train_tree_scores = cross_val_score(tree_reg, x_train, y_train,
                                   scoring="neg_mean_squared_error", cv=10)
train_tree_rmse_scores = np.sqrt(-train_tree_scores)
print("Training data root mean square error:", train_tree_rmse_scores.mean())

#Prediction on test set
test_tree_scores = cross_val_score(tree_reg, x_test, y_test,
                                  scoring="neg_mean_squared_error", cv=10)
test_tree_rmse_scores = np.sqrt(-test_tree_scores)
print("Test data root mean square error:", test_tree_rmse_scores.mean())

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#Create a Random Forest Regression based on the training data
print("RandomForest Regressor")
forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(x_train,y_train)

#Prediction on training set
train_forest_scores = cross_val_score(forest_reg, x_train, y_train,
                                      scoring="neg_mean_squared_error", cv=10)
train_forest_rmse_scores = np.sqrt(-train_forest_scores)
print("Training data root mean square error:",train_forest_rmse_scores.mean())

#Prediction on testing set
test_forest_scores = cross_val_score(forest_reg, x_test, y_test,
                                     scoring="neg_mean_squared_error", cv=10)
test_forest_rmse_scores = np.sqrt(-test_forest_scores)
print("Test data root mean square error:", test_forest_rmse_scores.mean())

In [ ]:
from sklearn.svm import SVR
#Same issue as above

gamma1, gamma2, gamma3 = 0.1, 0.01, 0.001
C1, C2, C3 = 1, 500, 1000
hyperparams = (gamma1, C1), (gamma1, C2), (gamma1, C3), (gamma2, C1), (gamma2, C2), (gamma2, C3), (gamma3, C1), (gamma3, C2), (gamma3, C3)  

from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score

for gamma, C in hyperparams:

    rbf_kernel_svRg_clf = SVR(kernel="rbf", gamma = gamma, C=C)
    rbf_kernel_svRg_clf.fit(x_train, y_train)

    #Performance metrics
    from sklearn.model_selection import cross_val_score

    svRg_mse = cross_val_score(svr_clf, x_test, y_test, cv=3, scoring="neg_mean_squared_error")

    svRg_rmse = np.sqrt(-svRg_mse)
    
    print("\n gamma is " + str(gamma))
    print("C is " + str(C))
    print("average:")
    svRg_average_error = np.mean(svRg_rmse)
    print(svRg_average_error)
    